# Analysis of Detected DOS Attacks
This notebook contains the analysis of data collected during the simulation and detection of Denial of Service (DOS) attacks. We will answer two questions:

1. Where are the suspect IP addresses located?
2. What is the most common attack?



In [ ]:
import pandas as pd

# Load the detected attacks from the CSV file
df = pd.read_csv('detected_attacks.csv')

# It shows us the firts rows of the file "detected_attacks" which contains the data"
df.head()



: 

In [ ]:
import requests

# Function to get the location of an IP address
def get_ip_location(ip):
    try:
        response = requests.get(f"http://ip-api.com/json/{ip}")
        data = response.json()
        return data['country'], data['lat'], data['lon']
    except:
        return "Unknown", None, None

# We apply the function to all detected IP address
df[['Location', 'lat', 'lon']] = df['Source IP'].apply(lambda ip: pd.Series(get_ip_location(ip)))

# Display the results
df[['Source IP', 'Location', 'lat', 'lon']].head()



In [ ]:
import folium

# Create a map centered around the average latitude and longitude
map = folium.Map(location=[20, 0], zoom_start=2)

# Add markers for each detected IP
for index, row in df.iterrows():
    if row['Location'] != "Unknown" and row['lat'] and row['lon']:
        folium.Marker([row['lat'], row['lon']], popup=row['Source IP']).add_to(map)

# Display the map
map



In [ ]:
import matplotlib.pyplot as plt

# Count the occurrences of each type of attack
attack_counts = df['Attack Type'].value_counts()

# Display the most frequent attacks
print(attack_counts)

# Create a bar chart of the most frequent attacks
attack_counts.plot(kind='bar')
plt.title('Most Frequent Attack Types')
plt.xlabel('Attack Type')
plt.ylabel('Number of Occurrences')
plt.show()



In [ ]:
# Analysis of the most frequent countries for suspect IPs
top_countries = df['Location'].value_counts().head(3).index.tolist()

# Analysis of the most common type of attack
most_common_attack = df['Attack Type'].value_counts().idxmax()

# Generate dynamic conclusion
conclusion = f"""
## Dynamic Conclusion

We have located suspect IP addresses mainly originating from the following countries: **{', '.join(top_countries)}**.

The most common detected attack type is **{most_common_attack}**.
"""

# Display the conclusion
from IPython.display import Markdown
Markdown(conclusion)



## Dynamic Conclusion

We have located suspect IP addresses mainly originating from the following countries: **France, USA, Canada**.

The most common detected attack type is **syn_ack**.
